In [7]:
import pandas as pd
import numpy as np
import io
import os
import re
import sys
from pandasql import sqldf
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
import jieba
root_path = 'C:\\Users\\luoyan011\\Desktop\\PersonalLearning\\GitHub\\NLP_data\\simplifyweibo_4_moods'
sys.path.append('C:\\Users\\luoyan011\\Desktop\\PersonalLearning\\GitHub\\python_functions\\jl_nlp_pkg')
stop_words_path = 'C:\\Users\\luoyan011\\Desktop\\PersonalLearning\\GitHub\\NLP\\recomend_system\\Chinese\\hit_stopwords.txt'

In [49]:
weibo_data = pd.read_csv(os.path.join(root_path, "simplifyweibo_4_moods.csv"))
weibo_sample = weibo_data.sample(10000)
weibo_sample.to_csv(os.path.join(root_path, 'weibo_sample_4mood.csv'), index=False)

In [50]:
import csv
import json
 
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        data = list(csvReader)
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))

make_json(os.path.join(root_path, "weibo_sample_4mood.csv"), os.path.join(root_path, "weibo_sample_4mood.json"))


Reference: https://blog.csdn.net/qq_42103091/article/details/119978834?utm_source=app&app_version=4.16.0

In [54]:
import pynlpir
def weibo_process(content):
    """
    功能：清洗微博内容并分词
    """
    processed_content = []
    # Replaces URLs with the word [URL]
    content = re.sub(r'(https?|ftp|file|www\.)[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]', '[URL]', content)
    # Replaces Email with the word [URL]
    content = re.sub(r'[a-zA-Z0-9_-]+@[a-zA-Z0-9_-]+[\.][a-zA-Z0-9_-]+', '[URL]', content)
    # Replaces user with the word FORWARD
    content = re.sub(r'(\/\/){0,1}@.*?(：|:| )', '[FORWARD]', content)
    # Replaces number  with the word [N]
    content = re.sub(r'\d+', '[N]', content)
    # Replace 2+ dots with space
    content = re.sub(r'[\.。…]{2,}', '。', content)
    # Replace 2+ ~~ 为 ~
    content = re.sub(r'~{2,}', '~', content)
    # Replace 2+ 叹号 为 一个叹号
    content = re.sub(r'[!！]{2,}', '!', content)
    # Replace 2+ 叹号 为 一个问号
    content = re.sub(r'[？?]{2,}', '?', content)
    # 去掉 //
    content = re.sub(r'//', ' ', content)
    # 去掉 引号
    content = re.sub(r'["“”\'‘’]', '', content)

    pynlpir.open(encoding='utf_8', encoding_errors='ignore')
    segments = pynlpir.segment(content, pos_tagging=False)
    i = 1
    count = len(segments) - 1
    for segment in segments:
        if re.match(r'\s+', segment):  # 过滤掉空格
            i = i + 1
            continue
        segment = re.sub(r'@[\S]+', '[USER_MENTION]', segment)
        processed_content.append(segment.strip())
        if (i == count) & (segment == '[USER_MENTION]'):  # 过滤掉最后一个单独的字
            break
        i = i + 1
    pynlpir.close()
    return processed_content


In [55]:
# license update for pynlpir: https://github.com/jinfeijoy/NLPIR/tree/master/License; data path: C:\Users\luoyan011\.conda\envs\nlp\Lib\site-packages\pynlpir\Data
weibo_process(weibo_data.review[0])

['啊呀',
 '呀',
 '！',
 '要',
 '死',
 '啦',
 '！',
 '么',
 '么',
 '么',
 '！',
 '只',
 '穿',
 '外套',
 '就',
 '好',
 '了',
 '，',
 '我',
 '认为',
 '里面',
 '那',
 '件',
 '很',
 '多',
 '余',
 '啊',
 '周小伦',
 '喜',
 '歡',
 '你',
 '各',
 '種',
 '五角星',
 '的',
 '項',
 '鏈',
 '和露',
 '胸',
 '的',
 '衣服',
 'XD',
 '伦',
 '伦',
 '啊',
 '，',
 '你',
 '真',
 '的',
 '是',
 '越来越',
 '帅',
 '了',
 '·',
 '·',
 '·',
 '·',
 '·']

In [56]:
import json
ignore_chars = ["/","@","【","】","#",":","[","]"]

def datasetProcess(org_path,save_path,stop_words):
    """
    功能：过滤出微博内容重点中文并进行分词
    """
    outcome = []
    with open(org_path,"r",encoding="utf-8") as fp:
        for idx,item in enumerate(json.load(fp)):
            print("processing item {}".format(idx))
            content = item.get("review")
            label = item.get("label")
            # content = "".join(regex.findall(chinese,content))
            seg_list = weibo_process(content)
            # seg_list = jieba.cut(content,cut_all=False)
            words = []
            for word in seg_list:
                if word in ignore_chars:
                    continue
                if word not in stop_words:
                    words.append(word)
            outcome.append({"content":words,"label":label})
    with open(save_path,"w",encoding="utf-8") as fp:
        json.dump(outcome,fp,ensure_ascii=False)


In [8]:
print('start read stopwords data.')
stopwords = []
with open(stop_words_path, 'r', encoding='utf-8') as f:
    for line in f:
        if len(line)>0:
            stopwords.append(line.strip())
print(stopwords)

start read stopwords data.
['———', '》），', '）÷（１－', '”，', '）、', '＝（', ':', '→', '℃', '&', '*', '一一', '~~~~', '’', '.', '『', '.一', './', '--', '』', '＝″', '【', '［＊］', '｝＞', '［⑤］］', '［①Ｄ］', 'ｃ］', 'ｎｇ昉', '＊', '//', '［', '］', '［②ｅ］', '［②ｇ］', '＝｛', '}', '，也', '‘', 'Ａ', '［①⑥］', '［②Ｂ］', '［①ａ］', '［④ａ］', '［①③］', '［③ｈ］', '③］', '１．', '－－', '［②ｂ］', '’‘', '×××', '［①⑧］', '０：２', '＝［', '［⑤ｂ］', '［②ｃ］', '［④ｂ］', '［②③］', '［③ａ］', '［④ｃ］', '［①⑤］', '［①⑦］', '［①ｇ］', '∈［', '［①⑨］', '［①④］', '［①ｃ］', '［②ｆ］', '［②⑧］', '［②①］', '［①Ｃ］', '［③ｃ］', '［③ｇ］', '［②⑤］', '［②②］', '一.', '［①ｈ］', '.数', '［］', '［①Ｂ］', '数/', '［①ｉ］', '［③ｅ］', '［①①］', '［④ｄ］', '［④ｅ］', '［③ｂ］', '［⑤ａ］', '［①Ａ］', '［②⑧］', '［②⑦］', '［①ｄ］', '［②ｊ］', '〕〔', '］［', '://', '′∈', '［②④', '［⑤ｅ］', '１２％', 'ｂ］', '...', '...................', '…………………………………………………③', 'ＺＸＦＩＴＬ', '［③Ｆ］', '」', '［①ｏ］', '］∧′＝［', '∪φ∈', '′｜', '｛－', '②ｃ', '｝', '［③①］', 'Ｒ．Ｌ．', '［①Ｅ］', 'Ψ', '－［＊］－', '↑', '.日', '［②ｄ］', '［②', '［②⑦］', '［②②］', '［③ｅ］', '［①ｉ］', '［①Ｂ］', '［①ｈ］', '［①ｄ］', '［①ｇ］', '［①②］', '［②ａ］', 'ｆ］', '

In [57]:
datasetProcess(os.path.join(root_path, "weibo_sample_4mood.json"),os.path.join(root_path, "weibo_sample_4mood_process.json"),stopwords)

processing item 0
processing item 1
processing item 2
processing item 3
processing item 4
processing item 5
processing item 6
processing item 7
processing item 8
processing item 9
processing item 10
processing item 11
processing item 12
processing item 13
processing item 14
processing item 15
processing item 16
processing item 17
processing item 18
processing item 19
processing item 20
processing item 21
processing item 22
processing item 23
processing item 24
processing item 25
processing item 26
processing item 27
processing item 28
processing item 29
processing item 30
processing item 31
processing item 32
processing item 33
processing item 34
processing item 35
processing item 36
processing item 37
processing item 38
processing item 39
processing item 40
processing item 41
processing item 42
processing item 43
processing item 44
processing item 45
processing item 46
processing item 47
processing item 48
processing item 49
processing item 50
processing item 51
processing item 52
pro

processing item 419
processing item 420
processing item 421
processing item 422
processing item 423
processing item 424
processing item 425
processing item 426
processing item 427
processing item 428
processing item 429
processing item 430
processing item 431
processing item 432
processing item 433
processing item 434
processing item 435
processing item 436
processing item 437
processing item 438
processing item 439
processing item 440
processing item 441
processing item 442
processing item 443
processing item 444
processing item 445
processing item 446
processing item 447
processing item 448
processing item 449
processing item 450
processing item 451
processing item 452
processing item 453
processing item 454
processing item 455
processing item 456
processing item 457
processing item 458
processing item 459
processing item 460
processing item 461
processing item 462
processing item 463
processing item 464
processing item 465
processing item 466
processing item 467
processing item 468


processing item 832
processing item 833
processing item 834
processing item 835
processing item 836
processing item 837
processing item 838
processing item 839
processing item 840
processing item 841
processing item 842
processing item 843
processing item 844
processing item 845
processing item 846
processing item 847
processing item 848
processing item 849
processing item 850
processing item 851
processing item 852
processing item 853
processing item 854
processing item 855
processing item 856
processing item 857
processing item 858
processing item 859
processing item 860
processing item 861
processing item 862
processing item 863
processing item 864
processing item 865
processing item 866
processing item 867
processing item 868
processing item 869
processing item 870
processing item 871
processing item 872
processing item 873
processing item 874
processing item 875
processing item 876
processing item 877
processing item 878
processing item 879
processing item 880
processing item 881


processing item 1243
processing item 1244
processing item 1245
processing item 1246
processing item 1247
processing item 1248
processing item 1249
processing item 1250
processing item 1251
processing item 1252
processing item 1253
processing item 1254
processing item 1255
processing item 1256
processing item 1257
processing item 1258
processing item 1259
processing item 1260
processing item 1261
processing item 1262
processing item 1263
processing item 1264
processing item 1265
processing item 1266
processing item 1267
processing item 1268
processing item 1269
processing item 1270
processing item 1271
processing item 1272
processing item 1273
processing item 1274
processing item 1275
processing item 1276
processing item 1277
processing item 1278
processing item 1279
processing item 1280
processing item 1281
processing item 1282
processing item 1283
processing item 1284
processing item 1285
processing item 1286
processing item 1287
processing item 1288
processing item 1289
processing it

processing item 1644
processing item 1645
processing item 1646
processing item 1647
processing item 1648
processing item 1649
processing item 1650
processing item 1651
processing item 1652
processing item 1653
processing item 1654
processing item 1655
processing item 1656
processing item 1657
processing item 1658
processing item 1659
processing item 1660
processing item 1661
processing item 1662
processing item 1663
processing item 1664
processing item 1665
processing item 1666
processing item 1667
processing item 1668
processing item 1669
processing item 1670
processing item 1671
processing item 1672
processing item 1673
processing item 1674
processing item 1675
processing item 1676
processing item 1677
processing item 1678
processing item 1679
processing item 1680
processing item 1681
processing item 1682
processing item 1683
processing item 1684
processing item 1685
processing item 1686
processing item 1687
processing item 1688
processing item 1689
processing item 1690
processing it

...

processing item 8407
processing item 8408
processing item 8409
processing item 8410
processing item 8411
processing item 8412
processing item 8413
processing item 8414
processing item 8415
processing item 8416
processing item 8417
processing item 8418
processing item 8419
processing item 8420
processing item 8421
processing item 8422
processing item 8423
processing item 8424
processing item 8425
processing item 8426
processing item 8427
processing item 8428
processing item 8429
processing item 8430
processing item 8431
processing item 8432
processing item 8433
processing item 8434
processing item 8435
processing item 8436
processing item 8437
processing item 8438
processing item 8439
processing item 8440
processing item 8441
processing item 8442
processing item 8443
processing item 8444
processing item 8445
processing item 8446
processing item 8447
processing item 8448
processing item 8449
processing item 8450
processing item 8451
processing item 8452
processing item 8453
processing it

processing item 8798
processing item 8799
processing item 8800
processing item 8801
processing item 8802
processing item 8803
processing item 8804
processing item 8805
processing item 8806
processing item 8807
processing item 8808
processing item 8809
processing item 8810
processing item 8811
processing item 8812
processing item 8813
processing item 8814
processing item 8815
processing item 8816
processing item 8817
processing item 8818
processing item 8819
processing item 8820
processing item 8821
processing item 8822
processing item 8823
processing item 8824
processing item 8825
processing item 8826
processing item 8827
processing item 8828
processing item 8829
processing item 8830
processing item 8831
processing item 8832
processing item 8833
processing item 8834
processing item 8835
processing item 8836
processing item 8837
processing item 8838
processing item 8839
processing item 8840
processing item 8841
processing item 8842
processing item 8843
processing item 8844
processing it

processing item 9191
processing item 9192
processing item 9193
processing item 9194
processing item 9195
processing item 9196
processing item 9197
processing item 9198
processing item 9199
processing item 9200
processing item 9201
processing item 9202
processing item 9203
processing item 9204
processing item 9205
processing item 9206
processing item 9207
processing item 9208
processing item 9209
processing item 9210
processing item 9211
processing item 9212
processing item 9213
processing item 9214
processing item 9215
processing item 9216
processing item 9217
processing item 9218
processing item 9219
processing item 9220
processing item 9221
processing item 9222
processing item 9223
processing item 9224
processing item 9225
processing item 9226
processing item 9227
processing item 9228
processing item 9229
processing item 9230
processing item 9231
processing item 9232
processing item 9233
processing item 9234
processing item 9235
processing item 9236
processing item 9237
processing it

processing item 9583
processing item 9584
processing item 9585
processing item 9586
processing item 9587
processing item 9588
processing item 9589
processing item 9590
processing item 9591
processing item 9592
processing item 9593
processing item 9594
processing item 9595
processing item 9596
processing item 9597
processing item 9598
processing item 9599
processing item 9600
processing item 9601
processing item 9602
processing item 9603
processing item 9604
processing item 9605
processing item 9606
processing item 9607
processing item 9608
processing item 9609
processing item 9610
processing item 9611
processing item 9612
processing item 9613
processing item 9614
processing item 9615
processing item 9616
processing item 9617
processing item 9618
processing item 9619
processing item 9620
processing item 9621
processing item 9622
processing item 9623
processing item 9624
processing item 9625
processing item 9626
processing item 9627
processing item 9628
processing item 9629
processing it

processing item 9983
processing item 9984
processing item 9985
processing item 9986
processing item 9987
processing item 9988
processing item 9989
processing item 9990
processing item 9991
processing item 9992
processing item 9993
processing item 9994
processing item 9995
processing item 9996
processing item 9997
processing item 9998
processing item 9999


In [58]:
f = open(os.path.join(root_path, "weibo_sample_4mood_process.json"), encoding="utf8")
sample_processed_json = json.load(f)

In [59]:
sample_processed_json

[{'content': ['很多',
   '作家',
   '孩子',
   '编',
   '故事',
   '成',
   '名家',
   '长',
   '袜子',
   '皮',
   '皮',
   '作者',
   '林格伦',
   '捏',
   '哈哈哈',
   '老',
   '江',
   '眼睛',
   '圆圆的',
   '瞪',
   '起来',
   '三',
   '角',
   '说',
   '爱',
   '讲',
   '圆圆的',
   '三角形',
   '变',
   '身',
   '不',
   '完全',
   '编成',
   '哲理',
   '故事',
   '哈哈哈',
   '讲',
   '圆圆的',
   '三角形',
   '~',
   '~',
   '~'],
  'label': '0'},
 {'content': ['邮件',
   '营销',
   '现在',
   '也许',
   '最',
   '便宜',
   '推广',
   '手段',
   '感觉',
   '卓越',
   '邮件',
   '营销',
   '很',
   '疯狂',
   '注册',
   '成',
   '会员',
   '后',
   '时不时',
   '发来',
   '促销',
   '邮件'],
  'label': '0'},
 {'content': ['看', '大师', '嘞', '苏秋萍', '老师', '中国', '元素', '大奖', '颁奖', '后'],
  'label': '0'},
 {'content': ['没', '办法', '爱', '死', '哈哈哈', '小家伙', '太', '可爱', '分享', '图片'],
  'label': '1'},
 {'content': ['权贵',
   '面目',
   '一览无遗',
   '老夫',
   '眼',
   '看穿',
   '少',
   '发',
   '淫威',
   '说',
   '见',
   '见',
   '人',
   '少',
   '献丑',
   '滚',
   '远',
   '点',
   '留',
   '张',
   '臭',
   '嘴',
   '主子